In [5]:
import os
import sys

sys.path.append('../txf_design-space/')
sys.path.append('../txf_design-space/flexibert')
sys.path.append('../protran/boshnas/boshnas/')
sys.path.append('../global_search/utils')
sys.path.append('../')

import re
import yaml
import json
import time
import torch
import shlex
import shutil
import argparse
import subprocess
import collections
import numpy as np
from tqdm import tqdm, tqdm_notebook

from embeddings.utils import graph_util, print_util as pu

sys.path.append('../txf_design-space/transformers/src/transformers')
import embedding_util

from boshnas import BOSHNAS
from acq import gosh_acq as acq

from transformers import BertModel
from transformers import RobertaTokenizer, RobertaModel
from transformers.models.bert.configuration_bert import BertConfig
from transformers.models.bert.modeling_modular_bert import BertModelModular, BertForMaskedLMModular, BertForSequenceClassificationModular

import warnings
warnings.filterwarnings("ignore")

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

from six.moves import cPickle as pickle
from scipy.interpolate import griddata
from sklearn.tree import plot_tree
from scipy.optimize import minimize, differential_evolution

PREFIX_CHECKPOINT_DIR = "checkpoint"

In [12]:
# Get Sobol samples and training status
glob_dataset_file = '../global_search/dataset/dataset.json'
models_dir = '../models/global_search/'

dataset = json.load(open(glob_dataset_file, 'r'))

print('MODEL\t\t\t\t\tTYPE\t\tITERATIONS')
for model in dataset.keys():
    model_dir = os.path.join(models_dir, model)
    
    # Finding the latest checkpoint for chosen model
    re_checkpoint = re.compile(r"^" + PREFIX_CHECKPOINT_DIR + r"\-(\d+)$")
    content = os.listdir(model_dir)
    checkpoints = [
            path
            for path in content
            if re_checkpoint.search(path) is not None and os.path.isdir(os.path.join(model_dir, path))]
    
    # Get latest checkpoint
    if len(checkpoints) > 0:
        checkpoint_dir = max(checkpoints, key=lambda x: int(re_checkpoint.search(x).groups()[0]))
        latest_checkpoint = int(checkpoint_dir.split('-')[1])
    else:
        latest_checkpoint = '-'
        
    print(f'{model}\t{dataset[model]["model_type"]}\t{latest_checkpoint}')

MODEL					TYPE		ITERATIONS
f36d5c40c770e4f787c78b43a2a09d78	shallow_narrow	1000000
e3ea4dd7dab98b7ca6aa439a22e2c669	deep_narrow	1000000
cc085bb5b433b6808e32bd7d1ff49584	deep_narrow	937000
bec578218d4f893fdd754d7497af18d5	deep_narrow	414500
2a8863139d4ccf931452c5f86c3a5a52	shallow_narrow	1000000
1d41e1cdd47ac30c570648bcddd22ad1	shallow_narrow	1000000
49ef529d1cada10c9b92463e129424d4	deep_narrow	907000
cbf26e711763f177f534ad38b8c7b8a7	shallow_narrow	-
dc7733ef95c734eee308504f5e0a384e	shallow_wide	298500
89ca0068830d26fdebfbc79a501b5f2a	deep_wide	-
fb76fc6b9cdde469653ce1d3f163b542	deep_wide	242000
fb04cff166ed53cb40500c78626889ea	deep_wide	182500
7cf1bfa8946458f7f6d261b11a9166bf	shallow_wide	-
081cb16979c43ce0b68a45650dc2de84	shallow_wide	350500
ae87fe8c1bbc496d0cfa8bf9e38c6657	deep_wide	-
9f4a1d6d19ccb2b1e070cf64ef5c45fe	shallow_wide	474000
